In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn import svm, datasets
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, accuracy_score
from sklearn.model_selection import cross_val_score, cross_val_predict
import warnings
warnings.filterwarnings('ignore')

In [2]:
dfis=pd.read_csv("data_with_grouping_operations_IS.csv")
dfos=pd.read_csv("data_with_grouping_operations_OS.csv")

In [89]:
dfis_sub=dfis[['year','condition_onehot_1',
       'condition_onehot_2', 'condition_onehot_3', 'condition_onehot_4',
       'condition_onehot_5', 'condition_onehot_6', 'condition_onehot_7','title_status_mean_price',
       'title_status_median_price', 'title_status_price_std',
       'transmission_mean_price', 'transmission_median_price',
       'transmission_price_std', 'type_mean_price', 'type_median_price',
       'type_price_std', 'manufacturer_make_mean_price',
       'manufacturer_make_median_price', 'manufacturer_make_price_std']]
dfos_sub=dfos[['year','condition_onehot_1',
       'condition_onehot_2', 'condition_onehot_3', 'condition_onehot_4',
       'condition_onehot_5', 'condition_onehot_6', 'condition_onehot_7','title_status_mean_price',
       'title_status_median_price', 'title_status_price_std',
       'transmission_mean_price', 'transmission_median_price',
       'transmission_price_std', 'type_mean_price', 'type_median_price',
       'type_price_std', 'manufacturer_make_mean_price',
       'manufacturer_make_median_price', 'manufacturer_make_price_std']]

In [90]:
for s in dfis_sub.columns:
    dfis_sub[s]=dfis_sub[s].fillna(dfis_sub[s].mean())
    dfos_sub[s]=dfos_sub[s].fillna(dfos_sub[s].mean())

In [91]:
num=20000
target=dfis.price.values[1:num]
data=dfis_sub.values[1:num,:]

In [95]:
rf_clf=RandomForestClassifier(max_depth=8, random_state=0)
#D_list=[4, 6, 8]
e_list=[2,3,4]
clf=GridSearchCV(rf_clf, {'min_samples_split':e_list},return_train_score=True)
clf.fit(data, target)
print(clf.best_params_)

{'min_samples_split': 4}


In [ ]:
def mae(estimator,X,y):
    pred=cross_val_predict(estimator, X,y)
    return abs(1-pred/y).mean()

In [52]:
rf_clf=RandomForestClassifier(n_estimators=100, max_depth=8, min_samples_split=2, max_features=None,random_state=0)
print(cross_val_score(rf_clf, data, target, cv = 3, scoring=mae).mean())

0.4014174204237377


# Type

In [41]:
pred_is=pd.DataFrame([], index=dfis.index, columns=['prediction'])
pred_os=pd.DataFrame([], index=dfos.index, columns=['prediction'])
type_list=dfis['type'].unique()
type_list=type_list[1:]
type_list

array(['pickup', 'truck', 'SUV', 'hatchback', 'van', 'sedan', 'coupe',
       'wagon', 'mini-van', 'other', 'convertible', 'bus', 'offroad'],
      dtype=object)

# Accuracy method

In [42]:
def accuracy(pred, y):
    return abs(1-pred/y).mean()

## NaN

In [93]:
rf_clf.fit(data, target)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=8, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [94]:
dfis_type=dfis[dfis['type'].isnull()]
dfis_type_sub=dfis_type[['year','condition_onehot_1',
       'condition_onehot_2', 'condition_onehot_3', 'condition_onehot_4',
       'condition_onehot_5', 'condition_onehot_6', 'condition_onehot_7','title_status_mean_price',
       'title_status_median_price', 'title_status_price_std',
       'transmission_mean_price', 'transmission_median_price',
       'transmission_price_std', 'type_mean_price', 'type_median_price',
       'type_price_std', 'manufacturer_make_mean_price',
       'manufacturer_make_median_price', 'manufacturer_make_price_std']]
    
dfos_type=dfos[dfos['type'].isnull()]
dfos_type_sub=dfos_type[['year','condition_onehot_1',
       'condition_onehot_2', 'condition_onehot_3', 'condition_onehot_4',
       'condition_onehot_5', 'condition_onehot_6', 'condition_onehot_7','title_status_mean_price',
       'title_status_median_price', 'title_status_price_std',
       'transmission_mean_price', 'transmission_median_price',
       'transmission_price_std', 'type_mean_price', 'type_median_price',
       'type_price_std', 'manufacturer_make_mean_price',
       'manufacturer_make_median_price', 'manufacturer_make_price_std']]
    
for s in dfis_type_sub.columns:
    dfis_type_sub[s]=dfis_type_sub[s].fillna(dfis[s].mean())
    dfos_type_sub[s]=dfos_type_sub[s].fillna(dfos[s].mean())
        
yestimate=rf_clf.predict(dfis_type_sub)
yestimate_os=rf_clf.predict(dfos_type_sub)
ytrue=dfis_type.price.values
print("accuracy: ",accuracy(yestimate,ytrue))

#Put into prediction
pred_is['prediction'].iloc[dfis[dfis['type'].isnull()].index]=yestimate
pred_os['prediction'].iloc[dfos[dfos['type'].isnull()].index]=yestimate_os

accuracy:  0.5366960771382376


## Other

In [44]:
for for t in type_list:
    print("Model of type: ", t)
    dfis_type=dfis[dfis['type']==t]
    dfis_type_sub=dfis_type[['year','condition_onehot_1',
       'condition_onehot_2', 'condition_onehot_3', 'condition_onehot_4',
       'condition_onehot_5', 'condition_onehot_6', 'condition_onehot_7','title_status_mean_price',
       'title_status_median_price', 'title_status_price_std',
       'transmission_mean_price', 'transmission_median_price',
       'transmission_price_std', 'type_mean_price', 'type_median_price',
       'type_price_std', 'manufacturer_make_mean_price',
       'manufacturer_make_median_price', 'manufacturer_make_price_std']]
    
    dfos_type=dfos[dfos['type']==t]
    dfos_type_sub=dfos_type[['year','condition_onehot_1',
       'condition_onehot_2', 'condition_onehot_3', 'condition_onehot_4',
       'condition_onehot_5', 'condition_onehot_6', 'condition_onehot_7','title_status_mean_price',
       'title_status_median_price', 'title_status_price_std',
       'transmission_mean_price', 'transmission_median_price',
       'transmission_price_std', 'type_mean_price', 'type_median_price',
       'type_price_std', 'manufacturer_make_mean_price',
       'manufacturer_make_median_price', 'manufacturer_make_price_std']]
    
    for s in dfis_type_sub.columns:
        dfis_type_sub[s]=dfis_type_sub[s].fillna(dfis_type_sub[s].mean())
        dfos_type_sub[s]=dfos_type_sub[s].fillna(dfos_type_sub[s].mean())
        
    num=min(10000,dfis_type.shape[0])
    target=dfis_type.price.values[1:num]
    data=dfis_type_sub.values[1:num,:]
    
    rf_clf=RandomForestClassifier(n_estimators=100, max_depth=8, min_samples_split=3, max_features=None,random_state=0)
    print("cross validation score: ",cross_val_score(rf_clf, data, target, cv = 3, scoring=mae).mean())
    
    #Use the random forest classifier
    rf_clf=RandomForestClassifier(n_estimators=100, max_depth=8, min_samples_split=3, max_features=None,random_state=0)
    rf_clf.fit(data,target)
    yestimate=rf_clf.predict(dfis_type_sub)
    yestimate_os=rf_clf.predict(dfos_type_sub)
    ytrue=dfis_type.price.values
    print("accuracy: ",accuracy(yestimate,ytrue))

    #Put into prediction
    pred_is['prediction'].iloc[dfis[dfis['type']==t].index]=yestimate
    pred_os['prediction'].iloc[dfos[dfos['type']==t].index]=yestimate_os

Model of type:  pickup
cross validation score:  0.3244188157978123
accuracy:  0.31810353995556495
Model of type:  truck
cross validation score:  0.39227751674339045
accuracy:  0.3669735020837369
Model of type:  SUV
cross validation score:  0.3727766116083991
accuracy:  0.34194288362984754
Model of type:  hatchback
cross validation score:  0.3150609144830749
accuracy:  0.24892412923921745
Model of type:  van
cross validation score:  0.38696080983385533
accuracy:  0.341846037920405
Model of type:  sedan
cross validation score:  0.3541785026995579
accuracy:  0.3255993801303746
Model of type:  coupe
cross validation score:  0.39102686093249156
accuracy:  0.38844916341098096
Model of type:  wagon
cross validation score:  0.31457956516799485
accuracy:  0.2647782734287451
Model of type:  mini-van
cross validation score:  0.3722783339435855
accuracy:  0.29635932245689767
Model of type:  other
cross validation score:  0.294947161803779
accuracy:  0.2669031463688367
Model of type:  convertible
c

# Output

In [96]:
pred_is.isnull().sum()

prediction    0
dtype: int64

In [97]:
pred_os.isnull().sum()

prediction    0
dtype: int64

In [99]:
pred_is.to_csv("pred_is.csv")
pred_os.to_csv("pred_os.csv")